# Chapter 9: Outputting modelling results

## Preparations

Load necessary packages.

In [ ]:
using DataFrames, JudiLing

Load the Dutch dataset we will be working with.

In [ ]:
dutch = JudiLing.load_dataset("../dat/dutch.csv");

Create `cue_obj` and `S` matrix (see tutorials on form and meaning representations):

In [ ]:
cue_obj = JudiLing.make_cue_matrix(dutch,
                                   grams=3,
                                   target_col="Ortho");

In [ ]:
S, words = JudiLing.load_S_matrix("../dat/dutch_w2v.csv"; header = false, sep = ",");

## Comprehension results

Compute comprehension mapping and predicted semantic matrix (see tutorial on comprehension and production):

In [ ]:
F = JudiLing.make_transform_matrix(cue_obj.C, S);
Shat = cue_obj.C * F;

Now we use the `write_comprehension_eval` function to save the results of our comprehension mapping to a comma-separated file. As can be seen in the following, the function requires the predicted semantic matrix `Shat`, the original semantic matrix `S`, the dataset `dutch`, the column with the target forms `:Ortho` and finally a path to the filename where we want to save the results. In this case, we save it in the `res` directory under the name `comp.csv`.

In [ ]:
JudiLing.write_comprehension_eval(Shat, S, dutch, :Ortho, "../res/comp.csv")

Now, we can examine the results by first loading the dataframe have just save as a `.csv` file:

In [ ]:
comp = JudiLing.load_dataset("../res/comp.csv");

In [ ]:
first(comp[:,2:5], 3)

## Production results

Compute production matrix and predicted form matrix (see tutorial on comprehension and production):

In [ ]:
G = JudiLing.make_transform_matrix(S, cue_obj.C);
Chat = S * G;

Now produce forms (see tutorial on producing forms):

In [ ]:
prod_res = JudiLing.learn_paths(dutch, cue_obj, S, F, Chat,
                                threshold = 0.01);

...and evaluate:

In [ ]:
JudiLing.eval_acc(prod_res, cue_obj)

We can again save the results to a `.csv` file. The `write2csv` function requires the output of `learn_paths`, `prod_res`, the dataset `dutch`, the `cue_obj` for the training and validation data (both are the same in the present case, see tutorial on cross validation for more information), the path to where the `.csv` file should be stored, and finally the column with the word forms in the dataset.

In [ ]:
JudiLing.write2csv(prod_res, dutch, cue_obj, cue_obj,
                   "../res/prod.csv", target_col=:Ortho)

We can again load the dataframe and inspect it.

In [ ]:
prod_df = JudiLing.load_dataset("../res/prod.csv");

In [ ]:
first(prod_df[:,[2,4,6,7,8,9]],3)

## Exercises

### Exercise 1

In [ ]:
latin = JudiLing.load_dataset("../dat/latin.csv")

cue_obj = JudiLing.make_cue_matrix(latin, grams=3, target_col=:Word);

S = JudiLing.make_S_matrix(
    latin,
    ["Lexeme"],
    ["Person", "Number", "Tense", "Voice", "Mood"],
    ncol=300)
JudiLing.display_matrix(latin, :Word, cue_obj, S, :S)

F = JudiLing.make_transform_matrix(cue_obj.C, S)
G = JudiLing.make_transform_matrix(S, cue_obj.C);

Shat = cue_obj.C * F
Chat = S * G

### Exercise 2

Writing comprehension results to a file.

In [ ]:
JudiLing.write_comprehension_eval(Shat, S, latin, :Word, "../res/comp_latin.csv")

Load file back into Julia

In [ ]:
comp_latin = JudiLing.load_dataset("../res/comp_latin.csv")

Which word has the highest support?

In [ ]:
argmax(comp_latin.support)

In [ ]:
comp_latin[61,:]

### Exercise 3

In [ ]:
prod_res = JudiLing.learn_paths(latin, cue_obj, S, F, Chat,
                                threshold = 0.1);

Write production evaluation results to a file:

In [ ]:
JudiLing.write2csv(prod_res, latin, cue_obj, cue_obj,
                   "../res/prod_latin.csv", target_col=:Word)

Load back into Julia

In [ ]:
prod_latin = JudiLing.load_dataset("../res/prod_latin.csv")

In [ ]:
# alternatively directly generate this dataframe with write2df:
prod_latin = JudiLing.write2df(prod_res, latin, cue_obj, cue_obj, target_col=:Word)

For which words was no form produced at all?

In [ ]:
prod_latin[ismissing.(prod_latin.pred),:]